In [1]:
using HDF5
using Plots
using LadderDGA
using Suppressor

path_to_input = "/home/julian/Hamburg/lDGA_shift_tests/data/50_100_s1_b12_u1.0"
path_to_output = "$(@__DIR__)/../../lDGA_test_runs"
cfg_file = "$(@__DIR__)/../../lDGA_test_runs/config_large.toml"

empty!(ARGS);
#push!(ARGS,path_to_input);
#push!(ARGS,path_to_output);
#include("../scripts/triqs_conv.jl");

┌ Info: Precompiling LadderDGA [78e40beb-bf89-4c0e-9d2b-bee278912f2b]
└ @ Base loading.jl:1317


Loading impurity quantities and constructing k grids.
removing the @supress block activates debugging output.

In [2]:
mP, sP, env, kGrids, qGrids, qGridLoc, freqRed_map, freqList, freqList_min, parents, ops, nFermi, nBose, shift, base, offset = readConfig(cfg_file)
kG = kGrids[1]
qG = qGrids[1]
νGrid, sumHelper_f, impQ_sp, impQ_ch, GImp_fft, GLoc_fft, Σ_loc, FUpDo, gImp, gLoc = setup_LDGA(kG, freqList, mP, sP, env);

┌ Info: Reading Inputs...
└ @ LadderDGA /home/julian/Hamburg/LadderDGA.jl/src/IO.jl:7
┌ Warning: TODO: check beta consistency, config <-> g0man, chi_dir <-> gamma dir
└ @ LadderDGA /home/julian/Hamburg/LadderDGA.jl/src/helpers.jl:82
┌ Info: Inputs Read. Starting Computation.
│ Found usable intervals for local susceptibility of length 
│   sp: 87
│   ch: 87 
│   χLoc_sp = 0.3364, χLoc_ch = 0.1361
└ @ LadderDGA /home/julian/Hamburg/LadderDGA.jl/src/helpers.jl:124


# Lambda charge solutions investigation

In [3]:
# "local" correction quantities
bubbleLoc = calc_bubble(νGrid, GImp_fft, qGridLoc, mP, sP);
locQ_sp = calc_χ_trilex(impQ_sp.Γ, bubbleLoc, qGridLoc, νGrid, sumHelper_f, mP.U, mP, sP);
locQ_ch = calc_χ_trilex(impQ_ch.Γ, bubbleLoc, qGridLoc, νGrid, sumHelper_f, -mP.U, mP, sP);
Σ_ladderLoc = calc_Σ(locQ_sp, locQ_ch, bubbleLoc, GImp_fft, FUpDo,
                     qGridLoc, sumHelper_f, mP, sP)
Σ_ladderLoc = Σ_ladderLoc .+ mP.n * mP.U/2.0;

LoadError: TaskFailedException

[91m    nested task error: [39mDimensionMismatch("cannot broadcast array to have fewer dimensions")
    Stacktrace:
      [1] [0m[1mcheck_broadcast_shape[22m[0m[1m([22m[90m#unused#[39m::[0mTuple[90m{}[39m, [90mAshp[39m::[0mTuple[90m{Base.OneTo{Int64}}[39m[0m[1m)[22m
    [90m    @ [39m[90mBase.Broadcast[39m [90m./[39m[90;4mbroadcast.jl:518[0m
      [2] [0m[1mcheck_broadcast_axes[22m
    [90m    @ [39m[90m./[39m[90;4mbroadcast.jl:523[0m[90m [inlined][39m
      [3] [0m[1mcheck_broadcast_axes[22m
    [90m    @ [39m[90m./[39m[90;4mbroadcast.jl:527[0m[90m [inlined][39m
      [4] [0m[1minstantiate[22m
    [90m    @ [39m[90m./[39m[90;4mbroadcast.jl:269[0m[90m [inlined][39m
      [5] [0m[1mmaterialize![22m
    [90m    @ [39m[90m./[39m[90;4mbroadcast.jl:894[0m[90m [inlined][39m
      [6] [0m[1mmaterialize![22m[0m[1m([22m[90mdest[39m::[0mComplexF64, [90mbc[39m::[0mBase.Broadcast.Br

In [4]:
# non local quantities 
bubble = calc_bubble(νGrid, GLoc_fft, qG, mP, sP);
nlQ_sp = calc_χ_trilex(impQ_sp.Γ, bubble, qG, νGrid, sumHelper_f, mP.U, mP, sP);
nlQ_ch = calc_χ_trilex(impQ_ch.Γ, bubble, qG, νGrid, sumHelper_f, -mP.U, mP, sP);

LoadError: TaskFailedException

[91m    nested task error: [39mDimensionMismatch("cannot broadcast array to have fewer dimensions")
    Stacktrace:
      [1] [0m[1mcheck_broadcast_shape[22m[0m[1m([22m[90m#unused#[39m::[0mTuple[90m{}[39m, [90mAshp[39m::[0mTuple[90m{Base.OneTo{Int64}}[39m[0m[1m)[22m
    [90m    @ [39m[90mBase.Broadcast[39m [90m./[39m[90;4mbroadcast.jl:518[0m
      [2] [0m[1mcheck_broadcast_axes[22m
    [90m    @ [39m[90m./[39m[90;4mbroadcast.jl:523[0m[90m [inlined][39m
      [3] [0m[1mcheck_broadcast_axes[22m
    [90m    @ [39m[90m./[39m[90;4mbroadcast.jl:527[0m[90m [inlined][39m
      [4] [0m[1minstantiate[22m
    [90m    @ [39m[90m./[39m[90;4mbroadcast.jl:269[0m[90m [inlined][39m
      [5] [0m[1mmaterialize![22m
    [90m    @ [39m[90m./[39m[90;4mbroadcast.jl:894[0m[90m [inlined][39m
      [6] [0m[1mmaterialize![22m[0m[1m([22m[90mdest[39m::[0mComplexF64, [90mbc[39m::[0mBase.Broadcast.Br

In [5]:
Σ_ladder = calc_Σ(nlQ_sp, nlQ_ch, bubble, GLoc_fft, FUpDo, qG, sumHelper_f, mP, sP)
Σ_ladder_corrected = Σ_ladder .- Σ_ladderLoc .+ Σ_loc[1:size(Σ_ladder,1)];

LoadError: UndefVarError: nlQ_sp not defined

In [6]:
plot(real(Σ_ladder[1:end-30,end]))
plot!(real(Σ_ladder_corrected[1:end-30,end]))

LoadError: UndefVarError: Σ_ladder not defined

In [7]:
plot(imag(Σ_ladder[1:30,end]))
plot!(imag(Σ_ladder_corrected[1:30,end]))

LoadError: UndefVarError: Σ_ladder not defined

In [8]:
sP.shift

true

In [9]:
sh_1 = LadderDGA.get_sum_helper(nlQ_sp.usable_ω, sP, :b)
χspw = LadderDGA.sum_freq(LadderDGA.kintegrate(qG, nlQ_sp.χ[nlQ_sp.usable_ω,:], dim=2)[:,1], [1], sh_1, mP.β)[1]

#plot(nlQ_sp.usable_ω, real(impQ_sp.χ_ω[impQ_sp.usable_ω]))
#plot!(nlQ_sp2.usable_ω, real(impQ_sp2.χ_ω[impQ_sp2.usable_ω]))

LoadError: UndefVarError: nlQ_sp not defined

In [10]:
λ_sp, λ_spch  = λ_correction!(impQ_sp, impQ_ch, FUpDo, Σ_loc, Σ_ladderLoc, nlQ_sp, nlQ_ch, bubble, GLoc_fft, qG, νGrid, mP, sP)

LoadError: UndefVarError: Σ_ladderLoc not defined

nothing: old=0.022919337508626597, new=[0.022919337508626434]

richardson: old=0.010033740564731804, new=[0.010033740564730212]


In [11]:
using Roots

get_sum_helper = LadderDGA.get_sum_helper
kintegrate = LadderDGA.kintegrate
sum_freq = LadderDGA.sum_freq
χ_λ(χ::AbstractArray, λ::Union{Float64}) = map(χi -> 1.0 / ((1.0 / χi) + λ), χ)
kGrid = qG
β = mP.β


sh = get_sum_helper(nlQ_ch.usable_ω, sP, :b)
χch_ω = kintegrate(kGrid, nlQ_ch.χ, dim=2)[:,1]
χch_sum = real(sum_freq(χch_ω[nlQ_ch.usable_ω], [1], sh, mP.β)[1])
rhs1 = mP.n * (1 - mP.n/2) - χch_sum
rhs2 = real(impQ_ch.χ_loc + impQ_sp.χ_loc - χch_sum)
usable_ω = intersect(nlQ_sp.usable_ω, nlQ_ch.usable_ω)
sh = get_sum_helper(usable_ω, sP, :b)
χupup_sum = sum_freq(kintegrate(kGrid, 0.5 .* (nlQ_ch.χ[usable_ω,:] .+ nlQ_sp.χ[usable_ω,:]), dim=2)[:,1], [1], sh, mP.β)[1]

println("usable range: $usable_ω\nχupup_nc=$χupup_sum")

χ_in = nlQ_sp.χ
usable_ω_i = nlQ_sp.usable_ω
χr    = real.(χ_in[usable_ω_i,:])
sh = get_sum_helper(usable_ω_i, sP, :b)
f(λint) = sum_freq(kintegrate(kGrid, χ_λ(χr, λint), dim=2)[:,1], [1], sh, β)[1] - rhs1
df(λint) = sum_freq(kintegrate(kGrid, -χ_λ(χr, λint) .^ 2, dim=2)[:,1], [1], sh, β)[1]
r2 = find_zeros(f, 0.02, 20.0, verbose=true)

println("lambda $r2")

LoadError: UndefVarError: nlQ_ch not defined